In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
from envs.env import *
from pathlib import Path
import pandas as pd
from config import settings
from model_architectures.curator import CuratorNetwork
import pickle
from contextual_ts_testing import *
from IPython.display import clear_output
from tqdm.notebook import tqdm
from constants import *

In [3]:
context_dim = 16
movie_dim = 24

## Prepare Environment

In [4]:
catalog_df = pd.read_parquet("data/whatson_catalog_df.parquet")
historical_data_df = pd.read_pickle("data/historical_data_df.pkl")
with open(settings.aud_model_dir/"rf_model.pkl", "rb") as f:    # load frozen viewership model
    audience_model = pickle.load(f)

In [5]:
env = TVProgrammingEnvironment(movie_catalog=catalog_df,
                         historical_df= historical_data_df,
                         audience_model=audience_model)

Setting up CompetitorDataManager...
Setting up Scalers...
Setting up RewardCalculator...
Setting up ContextualThompsonSampler...
Setting up CuratorNetwork...


In [6]:
date = '2025-08-06'
hour = 21

context_f, context, air_date = env.get_context_features_from_date_hour(date, hour)
env.get_available_movies(air_date, context)
recommended, top5_idx, top5_scores, w_tilde, movies, X_cands  = env.recommend_n_films(context, air_date)


Getting candidate features...


Movies:   0%|          | 0/1173 [00:00<?, ?it/s]

Done
31768: score = 0.619
754430: score = 0.607
342281: score = 0.606
291984: score = 0.598
21435: score = 0.594


## Recommendation loop

In [10]:
dates = create_date_list(10)

In [ ]:
#chosen_signals = input(prompt=f'If any factors were paricularly helpful in choosing the movie, please write the reward factor name \nChoose from {y_signal_feature_selection}')

In [12]:
status = 'go'
for date in tqdm(dates, desc="Dates"):
    if status == 'exit':
            break
    for hour in hours:
        if status == 'exit':
                break
        clear_output(wait=True)
        context_f, context, air_date = env.get_context_features_from_date_hour(date, hour)
        weekday = air_date.strftime("%A")
        day = air_date.strftime('%d')
        month = air_date.strftime('%B')
        year = air_date.strftime('%Y')

        print(f'Context: {hour, weekday, day, month, year}')
        env.get_available_movies(air_date, context)
        recommended, top5_idx, top5_scores, w_tilde, movies, X_cands  = env.recommend_n_films(context, air_date)
        env.show_top_breakdown(top5_idx, top5_scores, w_tilde, movies, X_cands)

        chosen_movie = '-1'
        while chosen_movie == '-1':
            chosen_movie = input(prompt='Choose a movie catalog id, if none are to your liking write -1 ')
            if chosen_movie != '-1':
                chosen_idx = movies.index(chosen_movie)
                chosen_signals = input(prompt=f'If any factors were paricularly helpful in choosing the movie, please write the reward factor name \nChoose from {y_signal_feature_selection}')
                if chosen_signals != '':
                    y_signals = env.encode_chosen_signals(chosen_signals)
                else:
                     y_signals = None
                print(y_signals)
            else:
                chosen_idx = -1
            
            for idx in top5_idx:
                x_i = X_cands[idx]                   # feature vector for this shown movie

                if idx == chosen_idx:
                    r = 1                            # curator chose it
                else:
                    r = 0                            # curator did *not* choose it

                
                env.cts.update(context_f, x_i, r, y_signals)                   # Bayesian update: A ← A + x xᵀ; b ← b + r·x
                
                param_std = np.sqrt(env.cts.expl_scale / env.cts.h_U)      # same shape as U
                print("Mean param std:", param_std.mean())
                print(f'h_U: {env.cts.h_U.mean()}, h_b: {env.cts.h_b.mean()}')
            fig, axes, samples = marginal_weight_distribution_cts(env.cts, env, num_samples=1000, show=False, bins=100)
            plt.show()
            print('Type go when done analyzing')
            status = input(prompt = 'Type go when done analyzing, type exit to break loop')
            if status == 'exit':
                break
            if chosen_movie == '-1':
                available_movies = [movie for  movie in env.available_movies if movie not in recommended]
                env.available_movies = available_movies
                print(len(env.available_movies))
                clear_output(wait=True)
                recommended, top5_idx, top5_scores, w_tilde, movies, X_cands  = env.recommend_n_films(context, air_date)
                print(f'Context: {hour, weekday, day, month, year}')
                env.show_top_breakdown(top5_idx, top5_scores, w_tilde, movies, X_cands)

        chosen_movie = '-1'

Context: (13, 'Thursday', '07', 'August', '2025')
Getting candidate features...


Movies:   0%|          | 0/1174 [00:00<?, ?it/s]

Done
44446: score = 0.596
24807: score = 0.582
27325: score = 0.577
105406: score = 0.569
3083: score = 0.557

🎬 44446: Angélique et le sultan  (total score = 0.596) (p = 0.644770968039597)

 Actors: Michele Mercier, Jean-Claude Pascal, Robert Hossein, Roger Pigaud, Ali Ben Ayed
  Breakdown:
    • curator_select_prob             0.805 ×  0.254 =   0.205 (+)
    • reward_audience                 0.716 ×  0.187 =   0.134 (+)
    • reward_novelty                  0.750 ×  0.152 =   0.114 (+)
    • reward_rights                   0.800 ×  0.137 =   0.109 (+)
    • reward_diversity                0.300 ×  0.115 =   0.034 (+)

🎬 24807: L'extravagant M. Deeds  (total score = 0.582) (p = 0.6415133023467826)

 Actors: Gary Cooper, Jean Arthur, George Bancroft, Lionel Stander, Douglas Dumbrille, Mayo Methot, Raymond Walburn
  Breakdown:
    • curator_select_prob             0.848 ×  0.254 =   0.216 (+)
    • reward_audience                 0.728 ×  0.187 =   0.136 (+)
    • reward_novelty       

ValueError: '' is not in list

## Save model

In [ ]:
env.cts.save(Path("models/ts_state_tick_match.npz"))